In [25]:
import pandas as pd
import os
import resource
import random
import numpy as np
import matplotlib as plt
from PIL import Image

The dataset has 2 folders, each with 15,000 images. We need to increase the maximum number of open files.

In [27]:
# Set the directory path
directory = 'data/Negative'

# Get a list of all the filenames in the directory
filenames = os.listdir(directory)

# Randomly select 1000 filenames from the list
sample_filenames = random.sample(filenames, 1000)

# Create an empty list to store the images
images = []

# Iterate over the sample filenames and load each image
for filename in sample_filenames:
    # Load the image file
    img = Image.open(os.path.join(directory, filename))

    # Append the image to the list
    images.append(img)

OSError: [Errno 24] Too many open files: 'data/Negative'